In [20]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('/Users/summer/desktop/notebook.mplstyle');
%config InlineBackend.figure_format = 'retina'
colors = ["#2078B5", "#FF7F0F", "#2CA12C", "#D72827", "#9467BE", "#8C574B",
            "#E478C2", "#808080", "#BCBE20", "#17BED0", "#AEC8E9", "#FFBC79", 
            "#98E08B", "#FF9896", "#C6B1D6", "#C59D94", "#F8B7D3", "#C8C8C8", 
           "#DCDC8E", "#9EDAE6"]

In [21]:
x = np.load('xlisttxt.npy')
y = np.load('ylisttxt.npy')

In [22]:
def lorenz96(xy,i,j):

    xy_dot[i] = (xy[0:I][(i-1)%I] * (xy[0:I][(i+1)%I] - xy[0:I][(i-2)%I]) - xy[0:I][i%I] + F - h*c/b*(np.sum([xy[I::][i::I]])))    
    xy_dot[I+j*I+i] = (c*b*xy[I:][((j+1)%J)*I+i]*(xy[I:][((j-1)%J)*I+i]-xy[I:][((j+2)%J)*I+i])-c*xy[I:][((j)%J)*I+i]+h*c/b*xy[0:I][i])

    return xy_dot

In [23]:
def RK4(xylist,step,i,j):
    
    oldxy = xylist[((I+J*I)*(step-1)):((I+J*I)*(step))]
    k1 = dt*lorenz96(oldxy,i,j)
    k2 = dt*lorenz96(oldxy + k1*0.5*dt,i,j)
    k3 = dt*lorenz96(oldxy + k2*0.5*dt,i,j)
    k4 = dt*lorenz96(oldxy + k3*dt,i,j)

    return oldxy + (k1 + 2*k2 + 2*k3 + k4)/6

In [24]:
def J4LE(xnow,x4jj,y4jj,i):   
      
    x4jj[(i)%I][(i)%I]  = (xnow[(i-1)%I] * (xnow[(i+1)%I] - xnow[(i-2)%I]) - 1 + F - h*c/b*(np.sum([y4jj[i::I]])))
    x4jj[(i)%I][(i-1)%I]  = (xnow[(i+1)%I] - xnow[(i-2)%I]) - xnow[i%I] + F - h*c/b*(np.sum([y4jj[i::I]]))
    x4jj[(i)%I][(i-2)%I]  = (xnow[(i-1)%I] * xnow[(i+1)%I] -  xnow[(i-2)%I]) - xnow[i%I] + F - h*c/b*(np.sum([y4jj[i::I]])) 
    x4jj[(i)%I][(i+1)%I]  = (xnow[(i-1)%I] - xnow[(i-1)%I] * xnow[(i-2)%I]) - xnow[i%I] + F - h*c/b*(np.sum([y4jj[i::I]])) 
    
    #y4jj[j*I+i] = c*b*y4jj[((j+1)%J)*I+i]*(y4jj[((j-1)%J)*I+i]-y4jj[((j+2)%J)*I+i])-c*y4jj[((j)%J)*I+i]+h*c/b 
                                  
    return x4jj                 

In [25]:
def RK4JJ(xnow,x4jj,y4jj,i,j):
    
    k1 = dt*J4LE(xnow,x4jj, y4jj, i, j)
    k2 = dt*J4LE(xnow,x4jj + k1*0.5*dt,y4jj, i, j)    
    k3 = dt*J4LE(xnow,x4jj + k2*0.5*dt,y4jj, i, j)
    k4 = dt*J4LE(xnow,x4jj + k3*dt ,y4jj, i, j)
    
    return x4jj + (k1 + 2*k2 + 2*k3 + k4)/6

In [26]:
F = 14
c = 10
b = 10
h = 1
dt = 0.0001
totstep = 200000
Jmax = 51
Imax = 51

In [30]:
#m = len(x1)
#N = 10000
#LE = np.zeros(m)
#v = x1
#LE = np.zeros((Jmax-4,Imax-4))

In [32]:
xjj = np.zeros((10-4,10-4))
print(xjj)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [ ]:
def LEcalc(x1,x4jj):

    q = np.identity(m)

    #JJ= J4LE(xy,i,j) unfinish
    #Jt = A*x    for DF1, A： current Df

    for i in range(N):

        DF1 = q = np.identity(m)
        
        for step in np.arange(1,totstep,1):
            
            #DF1 = RK4JJ(Jt,DF1,Df,i,j)
            #maybe should add time phase for x1 during x1 to parellal moving with JACOBIAN
    
        Z = DF1*q
        [q,r] = np.linalg.qr(Z)
        #LE[]= LE[] + np.log(np.amax(np.absolute(np.diag(r))))/(N)
        #LE = LE + log(abs(diag(r)))/N;

In [31]:
#def main():
for I in np.arange(4,Imax,1):
    for J in np.arange(8,Jmax,1):
        
        xjj = np.zeros((I-4,I-4))
        yjj = []
        xnow = np.zeros((I))
        
        xylist = np.zeros([(totstep)*(I+J*I)])
        #s=0
        newx=[]
        newlist = []
        #time=[0]
        
        for ii in range(I):
            xylist[ii] = x[ii]
        newx.append([x[ii] for ii in range(len(x))])
        #print(newx)

        for jjj in range(J):
            for iii in range(I):
                xylist[I + jjj*I + iii] = y[jjj,iii]
                
        for step in np.arange(1,totstep,1):
            xy_dot= np.zeros((I+J*I))
            for i in range(I):
                #s+=1
                #time.append(s)
                for j in range(J):
                    xylist[(I+J*I)*(step):(I+J*I)*(step+1)] = RK4(xylist,step, i,j) 
                    newlist = xylist[(I+J*I)*(step):(I+J*I)*(step+1)][0:I]     
                #newx.append([newlist[ij] for ij in range(len(newlist))])
        
            for a in range(I):
                xnow[a] = (xylist[(I+J*I)*(step-1):(I+J*I)*(step)][0:I])[a]
                xjj[a:] = (xylist[(I+J*I)*(step-1):(I+J*I)*(step)][0:I])[a]
                yjj.append(xylist[(I+J*I)*(step-1):(I+J*I)*(step)][I:]) 
    
                xjj = J4LE(xnow,xjj,yjj,i) # at this time
                newx.append([newlist[ij] for ij in range(len(newlist))])
                
        ####### use def LEcalc(x1,x4jj)'s function

               
                
                
#LEcalc(f,Df,x(:))  # call
#LE = LEcalc(f,Df, v0) #def

[[0.06326007914970078, 0.9114241431540494, 0.031216996094494465, 0.7479707658277304]]


NameError: name 'xnow' is not defined

In [ ]:
print(LE)          
plt.imshow(LE, cmap=plt.cm.jet)
cb = plt.colorbar(label='LE', pad=0.1)
plt.axes().set_aspect('equal')
plt.show()